In [1]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
! pip install transformers==4.27.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 9.6 MB/s eta 0:00:00


In [4]:
! pip install torchtext==0.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 1.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.9.1 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.9.1 which is incompatible.


In [5]:
import torch
device = torch.device("cuda")
torch.cuda.init()

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
from transformers import GPT2Tokenizer, GPT2Config, GPT2LMHeadModel, TextDataset

In [8]:
model_path = "/content/gdrive/My Drive/Project/model/chitchat_generator.pt"

In [9]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [10]:
model.load_state_dict(torch.load(model_path, map_location=device))

<All keys matched successfully>

In [11]:
import re
def getResponse(input_text, model,tokenizer, device):
  input_ids = tokenizer.encode(input_text, return_tensors='pt')
  input_ids = input_ids.to(device)
  model = model.to(device)
  attention_mask = torch.LongTensor([1] * len(input_ids))
  output_ids = model.generate(input_ids,pad_token_id=tokenizer.eos_token_id, max_length=50, num_beams=5, 
                                      num_return_sequences=3, 
                                      no_repeat_ngram_size=2, 
                                      early_stopping=True)
  output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  bot_response = re.search(r'Bot: (.+)', output_text).group(1)

  return bot_response

In [12]:
with open('/content/gdrive/My Drive/Project/dataset/combined_test.txt', 'r') as f:
    test_data = f.readlines()

In [13]:
type(test_data[0])

str

In [29]:
def prepare(test_data):
  input = {}
  response = {}
  conv_id = 1
  idx = 1
  for i in range(len(test_data)):
    input[idx]=""
    response[idx]=""
    if(test_data[i]=="\n" or test_data[i]=='\n'):
      idx+=1
    for data in test_data:
      if(conv_id>=idx):
        break
      if(data.startswith("User")):
        input[conv_id]+=data
      elif(data.startswith("Bot")):
        response[conv_id]+=data
      else:
        conv_id+=1
  return input,response

In [30]:
input, response = prepare(test_data)

In [31]:
len(input), len(response)

(11931, 11931)

In [32]:
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu
def getBLEUScore(input, response, model,tokenizer, device):
  generated_response = {}
  for idx, text in tqdm(input.items()):
    generated_response[idx] = getResponse(text, model,tokenizer, device)
  bleu_score = corpus_bleu(generated_response.values, response.values)
  return bleu_score

In [33]:
input_ids = tokenizer.encode("Hello, I am very sad", return_tensors='pt')
input_ids = input_ids.to(device)
model = model.to(device)
output_ids = model.generate(input_ids,pad_token_id=tokenizer.eos_token_id, max_length=50, num_beams=5, 
                                      num_return_sequences=3, 
                                      no_repeat_ngram_size=2, 
                                      early_stopping=True)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
bot_response = re.search(r'Bot: (.+)', output_text).group(1).strip()
output_text, bot_response

('Hello, I am very sad to hear that.\nBot: Oh no_comma_ what happened?\n\nUser: I went to the store and bought a new car.  I was so mad. I had no idea what to do',
 'Oh no_comma_ what happened?')

In [ ]:
bleu_score = getBLEUScore(input, response, model, tokenizer, device)
bleu_score

  3%|▎         | 400/11931 [02:29<1:20:19,  2.39it/s]

In [21]:
device

device(type='cuda')